In [1]:
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from sklearn.metrics import accuracy_score
from tensorflow import keras
from sklearn import metrics
import tensorflow as tf
import pandas as pd
import numpy as np
import plotting
import hls4ml
import shap
import sys
import os

np.random.seed(77)

import matplotlib.pyplot as plt
import mplhep
plt.style.use(mplhep.style.CMS)


def print_dict(d, indent=0):
    align=20
    for key, value in d.items():
        print('  ' * indent + str(key), end='')
        if isinstance(value, dict):
            print()
            print_dict(value, indent+1)
        else:
            print(':' + ' ' * (20 - len(key) - 2 * indent) + str(value))

2023-07-08 12:37:09.436304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/exp_soft/llr/python/3.9.9/lib/vtk:/opt/exp_soft/llr/python/3.9.9/lib:/usr/lib64/classads:/usr/lib64:/usr/lib
2023-07-08 12:37:09.436338: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/llr/cms/motta/.local/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes,

In [2]:
os.environ['PATH'] = '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vivado/2020.1/bin:' + os.environ['PATH']
!source /data/elec_soft/Xilinx_Software/Soft_2020.1/Vivado/2020.1/settings64.sh
os.environ['PATH'] = '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/bin:' + os.environ['PATH']
!source /data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/settings64.sh

In [3]:
options = {
    'v'          : '22QNTZD',
    'date'       : '2023_07_06',
    'caloClNxM'  : '5x9',
    'sparsityID'   : 0.25,
    'sparsityCAL'   : 0.70,
}

In [4]:
indir = '/data_CMS/cms/motta/Phase2L1T/'+options['date']+'_v'+options['v']+'/TauMinator_CE_cltw'+options['caloClNxM']+'_Training'

N = int(options['caloClNxM'].split('x')[0])
M = int(options['caloClNxM'].split('x')[1])

sparsityTag_id = '_QNTZD_sparsity'+str(options['sparsityID'])
sparsityTag_cal = '_QNTZD_sparsity'+str(options['sparsityCAL'])

**Only models with one input tensor are currently supported by VivadoAcceleratorBackend.
Therefore, the CNN model is in this case a special version that takes only the image as an input and gives as an output the flattened tensor out of the convolutions withou concatenating the position of the cluster.**

In [5]:
# quantized and pruned models
QCNN = keras.models.load_model(indir+'/CNNmodel'+sparsityTag_id+'_special4accelerator', compile=False)
QDNNident = keras.models.load_model(indir+'/ID_DNNmodel'+sparsityTag_id, compile=False)
QDNNcalib = keras.models.load_model(indir+'/CAL_DNNmodel'+sparsityTag_cal, compile=False)

2023-07-08 12:37:16.132242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /grid_mnt/opt__exp_soft/llr/python/3.9.9/lib/python3.9/site-packages/cv2/../../lib64:/opt/exp_soft/llr/python/3.9.9/lib/vtk:/opt/exp_soft/llr/python/3.9.9/lib:/usr/lib64/classads:/usr/lib64:/usr/lib
2023-07-08 12:37:16.132280: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-08 12:37:16.132307: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (llrgrhgtrig.in2p3.fr): /proc/driver/nvidia/version does not exist
2023-07-08 12:37:16.132637: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical

# QUANTIZED MODELS ONLY

In [6]:
############################## Pass quantized CNN model through hls4ml ##############################

compl = True
synth = True
if compl:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    QCNN_hls_cfg = hls4ml.utils.config_from_keras_model(QCNN, granularity='name')
    QCNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    QCNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in QCNN_hls_cfg['LayerName'].keys():
        QCNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Latency'
        QCNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        QCNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    QCNN_hls_cfg['LayerName']['TowerClusterImage']['Precision'] = 'ap_ufixed<10,8>'

    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['ParallelizationFactor'] = 4
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['weight'] = 'ap_fixed<5,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['bias'] = 'ap_fixed<6,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['accum'] = 'ap_fixed<14,7>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['result'] = 'ap_fixed<14,7>'

    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer1']['Precision']['accum'] = 'ap_fixed<10,7>'
    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer1']['Precision']['result'] = 'ap_fixed<9,6>'

    QCNN_hls_cfg['LayerName']['MP_CNNpBNlayer1']['Precision'] = 'ap_fixed<9,6>'

    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['weight'] = 'ap_fixed<5,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['bias'] = 'ap_fixed<6,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['accum'] = 'ap_fixed<14,6>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['result'] = 'ap_fixed<13,5>'

    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer2']['Precision']['result'] = 'ap_fixed<8,5>'
    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer2']['Precision']['accum'] = 'ap_fixed<8,5>'

    print(QCNN_hls_cfg)

    QCNN_hls = hls4ml.converters.convert_from_keras_model(QCNN,
                                                          hls_config=QCNN_hls_cfg,
                                                          output_dir=indir+'/CNNmodel_HLS_XCU200deployment/',
                                                          backend='VivadoAccelerator',
                                                          board='alveo-u200',
                                                          io_type='io_parallel',
                                                          platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QCNN_hls.compile()

if synth:
    QCNN_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=True)

Interpreting Model
Topology:
Layer name: TowerClusterImage, layer type: Input
Layer name: CNNpBNlayer1, layer type: QConv2DBatchnorm
  -> Activation (linear), layer name: CNNpBNlayer1
Layer name: RELU_CNNpBNlayer1, layer type: QActivation
  -> Activation (quantized_relu(10,7)), layer name: RELU_CNNpBNlayer1
Layer name: MP_CNNpBNlayer1, layer type: MaxPooling2D
Layer name: CNNpBNlayer2, layer type: QConv2DBatchnorm
  -> Activation (linear), layer name: CNNpBNlayer2
Layer name: RELU_CNNpBNlayer2, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CNNpBNlayer2
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'TowerClusterImage': {'Precision': 'ap_ufixed<10,8>', 'Strategy': 'Latency', 'ReuseFactor': 1, 'Trace': True}, 'CNNpBNlayer1': {'Precision': {'weight': 'ap_fixed<5,1>', 'bias': 'ap_fixed<6,1>', 'accum': 'ap_fixed<14,7>', 'result': 'ap_fixed<14,7>'}, 'ReuseFactor': 1, 'Strategy': 'Latency', 'Trace': True, 'Pa

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'PixelLoop' (firmware/nnet_utils/nnet_conv2d_latency.h:202) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 4.
INFO: [HLS 200-489] Unrolling loop 'Product1' (firmware/nnet_utils/nnet_conv2d_latency.h:208) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 8.
INFO: [HLS 200-489] Unrolling loop 'Product2' (firmware/nnet_utils/nnet_conv2d_latency.h:210) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 4.
INFO: [HLS 200-489] Unrolling loop 'ResetAccum' (firmware/nnet_utils/nnet_conv2d_latency.h:216) in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mo

INFO: [XFORM 203-11] Balancing expressions in function 'nnet::fill_buffer_2<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>::fill_buffer' (firmware/nnet_utils/nnet_code_gen.h:38)...13 expression(s) balanced.
INFO: [XFORM 203-11] Balancing expressions in function 'nnet::conv_2d_cl<ap_ufixed<10, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<14, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' (firmware/nnet_utils/nnet_conv2d_latency.h:75:59)...88 expression(s) balanced.
INFO: [XFORM 203-11] Balancing expressions in function 'nnet::conv_2d_cl<ap_fixed<9, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<13, 5, (ap_q_mode)5, (ap_o_mode)3, 0>, config5>' (firmware/nnet_utils/nnet_conv2d_latency.h:75:59)...63 expression(s) balanced.
INFO: [HLS 200-111] Finished Pre-synthesis Time (s): cpu = 00:00:46 ; elapsed = 00:00:49 . Memory (MB): peak = 1824.188 ; gain = 1295.879 ; free physical = 43301 ; free virtual = 333941
INFO: [HLS 200-111] Finished Architecture Synthesis Time (s): cpu = 00:01:02 ; e

INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable lifetime analysis.
INFO: [BIND 205-101] Exploring resource sharing.
INFO: [BIND 205-101] Binding ...
INFO: [BIND 205-100] Finished micro-architecture generation.
INFO: [HLS 200-111]  Elapsed time: 0.23 seconds; current allocated memory: 627.218 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'conv_2d_cl_ap_fixed_ap_fixed_13_5_5_3_0_config5_s' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-61] Pipelining loop 'PartitionLoop'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 2.
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 1.15 seconds; current allocated memory: 629.315 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIN

...

Synthesis Optimization Runtime : Time (s): cpu = 00:01:36 ; elapsed = 00:03:09 . Memory (MB): peak = 3377.492 ; gain = 1120.551 ; free physical = 40014 ; free virtual = 331024
Synthesis Optimization Complete : Time (s): cpu = 00:01:36 ; elapsed = 00:03:09 . Memory (MB): peak = 3377.492 ; gain = 1120.551 ; free physical = 40021 ; free virtual = 331024
INFO: [Project 1-571] Translating synthesized netlist
Netlist sorting complete. Time (s): cpu = 00:00:00.23 ; elapsed = 00:00:00.24 . Memory (MB): peak = 3385.699 ; gain = 0.000 ; free physical = 39891 ; free virtual = 330894
INFO: [Netlist 29-17] Analyzing 1499 Unisim elements for replacement
INFO: [Netlist 29-28] Unisim Transformation completed in 0 CPU seconds
INFO: [Project 1-570] Preparing netlist for logic optimization
INFO: [Opt 31-138] Pushed 0 inverter(s) to 0 load pin(s).
Netlist sorting complete. Time (s): cpu = 00:00:00.01 ; elapsed = 00:00:00 . Memory (MB): peak = 3448.566 ; gain = 0.000 ; free physical = 39693 ; free virtual 

INFO: [IP_Flow 19-2181] Payment Required is not set for this core.
INFO: [IP_Flow 19-2187] The Product Guide file is missing.
INFO: [Ipptcl 7-1486] check_integrity: Integrity check passed.
INFO: [Common 17-206] Exiting Vivado at Sat Jul  8 12:46:05 2023...
Option Map File Used: '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/data/vitis/vpp/optMap.xml'

****** v++ v2020.1 (64-bit)
  **** SW Build 2902540 on Wed May 27 19:54:35 MDT 2020
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ link can be found at:
	Reports: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CNNmodel_HLS_XCU200deployment/xclbin_files/_x/reports/link
	Log files: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CNNmodel_HLS_XCU200deployment/xclbin_files/_x/logs/link
INFO: [v++ 60-1657] Initializing dispatch client.
Running Dispat

INFO: [CF2BD 82-28] cf2xd finished successfully
INFO: [CF2BD 82-31] Launching cf_xsd: cf_xsd -disable-address-gen -bd pfm_dynamic.bd -dn dr -dp /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/sys_link/_sysl/.xsd
INFO: [CF2BD 82-28] cf_xsd finished successfully
INFO: [SYSTEM_LINK 82-37] [12:46:49] cf2bd finished successfully
Time (s): cpu = 00:00:02 ; elapsed = 00:00:02 . Memory (MB): peak = 1402.066 ; gain = 0.000 ; free physical = 42422 ; free virtual = 333124
INFO: [v++ 60-1441] [12:46:49] Run run_link: Step system_link: Completed
Time (s): cpu = 00:00:12 ; elapsed = 00:00:13 . Memory (MB): peak = 1436.730 ; gain = 0.000 ; free physical = 42453 ; free virtual = 333149
INFO: [v++ 60-1443] [12:46:49] Run run_link: Step cf2sw: Started
INFO: [v++ 60-1453] Command Line: cf2sw -sdsl /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CNNmodel_

[13:16:03] Finished 2nd of 6 tasks (FPGA linking synthesized kernels to platform). Elapsed time: 00h 29m 09s 

[13:16:03] Starting logic optimization..
[13:17:36] Phase 1 Retarget
[13:17:36] Phase 2 Constant propagation
[13:18:07] Phase 3 Sweep
[13:18:38] Phase 4 BUFG optimization
[13:18:38] Phase 5 Shift Register Optimization
[13:19:08] Phase 6 Post Processing Netlist
[13:20:10] Finished 3rd of 6 tasks (FPGA logic optimization). Elapsed time: 00h 04m 07s 

[13:20:10] Starting logic placement..
[13:20:41] Phase 1 Placer Initialization
[13:20:41] Phase 1.1 Placer Initialization Netlist Sorting
[13:22:14] Phase 1.2 IO Placement/ Clock Placement/ Build Placer Device
[13:23:46] Phase 1.3 Build Placer Netlist Model
[13:25:19] Phase 1.4 Constrain Clocks/Macros
[13:25:19] Phase 2 Global Placement
[13:25:19] Phase 2.1 Floorplanning
[13:25:50] Phase 2.1.1 Partition Driven Placement
[13:25:50] Phase 2.1.1.1 PBP: Partition Driven Placement
[13:26:21] Phase 2.1.1.2 PBP: Clock Region Placement
[13:

XRT Build Version: 2.6.0 (Vitis)
       Build Date: 2020-05-07 15:30:09
          Hash ID: 9d35aca9e6be09a5402ec036a5607d26e74e01cc
Creating a default 'in-memory' xclbin image.

Section: 'DEBUG_IP_LAYOUT'(9) was successfully added.
Size   : 440 bytes
Format : JSON
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/debug_ip_layout.rtd'

Section: 'BITSTREAM'(0) was successfully added.
Size   : 41449582 bytes
Format : RAW
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/partial.bit'

Section: 'MEM_TOPOLOGY'(6) was successfully added.
Format : JSON
File   : 'mem_topology'

Section: 'IP_LAYOUT'(8) was successfully added.
Format : JSON
File   : 'ip_layout'

Section: 'CONNECTIVITY'(7) was successfully added.
Format : JSON
File   : 'connectivity'

Section: 'CLOCK_FREQ_T

In [7]:
############################## Pass quantized identification DNN model through hls4ml ##############################

compl = True
synth = True
if compl:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    id_QDNN_hls_cfg = hls4ml.utils.config_from_keras_model(QDNNident, granularity='name')
    id_QDNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    id_QDNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in id_QDNN_hls_cfg['LayerName'].keys():
        id_QDNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Resources'
        id_QDNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        id_QDNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    id_QDNN_hls_cfg['LayerName']['middleMan']['Precision'] = 'ap_fixed<11,6>'

    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['weight'] = 'ap_fixed<3,1>'
    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['accum'] = 'ap_fixed<13,6>'
    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['result'] = 'ap_fixed<10,6>'
    id_QDNN_hls_cfg['LayerName']['RELU_IDlayer1']['Precision'] = 'ap_fixed<9,6>'

    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['weight'] = 'ap_fixed<3,1>'
    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['accum'] = 'ap_fixed<11,6>'
    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['result'] = 'ap_fixed<10,5>'
    id_QDNN_hls_cfg['LayerName']['RELU_IDlayer2']['Precision'] = 'ap_fixed<9,5>'

    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['weight'] = 'ap_fixed<2,1>'
    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['accum'] = 'ap_fixed<7,4>'
    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['result'] = 'ap_fixed<7,4>'

    id_QDNN_hls_cfg['LayerName']['sigmoid_IDout']['Precision'] = 'ap_fixed<8,1>'
    id_QDNN_hls_cfg['LayerName']['sigmoid_IDout']['Strategy'] = 'Stable'

    print(id_QDNN_hls_cfg)

    QDNNident_hls = hls4ml.converters.convert_from_keras_model(QDNNident,
                                                               hls_config=id_QDNN_hls_cfg,
                                                               output_dir=indir+'/ID_DNNmodel_HLS_XCU200deployment/',
                                                               backend='VivadoAccelerator',
                                                               board='alveo-u200',
                                                               io_type='io_parallel',
                                                               platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QDNNident_hls.compile()

if synth:
    QDNNident_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=True)

Interpreting Model
Topology:
Layer name: middleMan, layer type: Input
Layer name: IDlayer1, layer type: QDense
  -> Activation (linear), layer name: IDlayer1
Layer name: RELU_IDlayer1, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_IDlayer1
Layer name: IDlayer2, layer type: QDense
  -> Activation (linear), layer name: IDlayer2
Layer name: RELU_IDlayer2, layer type: QActivation
  -> Activation (quantized_relu(8,5)), layer name: RELU_IDlayer2
Layer name: IDout, layer type: QDense
  -> Activation (linear), layer name: IDout
Layer name: sigmoid_IDout, layer type: Activation
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'middleMan': {'Precision': 'ap_fixed<11,6>', 'Strategy': 'Resources', 'ReuseFactor': 1, 'Trace': True}, 'IDlayer1': {'Precision': {'weight': 'ap_fixed<3,1>', 'accum': 'ap_fixed<13,6>', 'result': 'ap_fixed<10,6>'}, 'ReuseFactor': 1, 'Strategy': 'Resources', 'Trace': True}, 'IDlayer1_linear': 

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'ResetAccum' (firmware/nnet_utils/nnet_dense_latency.h:68) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 16.
INFO: [HLS 200-489] Unrolling loop 'Accum1' (firmware/nnet_utils/nnet_dense_latency.h:73) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 34.
INFO: [HLS 200-489] Unrolling loop 'Accum2' (firmware/nnet_utils/nnet_dense_latency.h:74) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 16.
INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_dense_latency.h:81) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3

INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>.0.0.0.0.0.0' by setting 'biases[4].V' to 'b2.V.4'.
INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>.0.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>.0.0.0.0.0.0.0' by setting 'biases[5].V' to 'b2.V.5'.
INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<10, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>.0.0.0

INFO: [HLS 200-111] Finished Architecture Synthesis Time (s): cpu = 00:00:54 ; elapsed = 00:00:58 . Memory (MB): peak = 1824.188 ; gain = 1295.879 ; free physical = 49724 ; free virtual = 338761
INFO: [HLS 200-10] Starting hardware synthesis ...
INFO: [HLS 200-10] Synthesizing 'myproject_axi' ...
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'dense_latency_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-61] Pipelining function 'dense_latency.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 2.
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 58.56 seconds; current allocated memory: 462.426 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performi

...

# set_property  ip_repo_paths  ${myproject}_prj [current_project]
# update_ip_catalog
INFO: [IP_Flow 19-234] Refreshing IP repositories
INFO: [IP_Flow 19-1700] Loaded user IP repository '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/myproject_prj'.
INFO: [IP_Flow 19-2313] Loaded Vivado IP repository '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vivado/2020.1/data/ip'.
update_ip_catalog: Time (s): cpu = 00:00:02 ; elapsed = 00:00:07 . Memory (MB): peak = 2256.340 ; gain = 0.000 ; free physical = 49886 ; free virtual = 338928
# add_files -scan_for_includes {src/krnl_rtl_int.sv src/krnl_rtl_axi_read_master.sv src/krnl_rtl_counter.sv src/myproject_kernel.v src/krnl_rtl_axi_write_master.sv src/krnl_rtl_control_s_axi.v}
# import_files {src/krnl_rtl_int.sv src/krnl_rtl_axi_read_master.sv src/krnl_rtl_counter.sv src/myproject_kernel.v src/krnl_rtl_axi_write_master.sv src/krnl_rtl_control_s_axi.v}
# create_ip -

INFO: [IP_Flow 19-2181] Payment Required is not set for this core.
INFO: [IP_Flow 19-2187] The Product Guide file is missing.
INFO: [Ipptcl 7-1486] check_integrity: Integrity check passed.
INFO: [Common 17-206] Exiting Vivado at Sat Jul  8 14:29:52 2023...
Option Map File Used: '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/data/vitis/vpp/optMap.xml'

****** v++ v2020.1 (64-bit)
  **** SW Build 2902540 on Wed May 27 19:54:35 MDT 2020
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ link can be found at:
	Reports: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/reports/link
	Log files: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/logs/link
INFO: [v++ 60-1657] Initializing dispatch client.
Running 

INFO: [CF2BD 82-28] cf2xd finished successfully
INFO: [CF2BD 82-31] Launching cf_xsd: cf_xsd -disable-address-gen -bd pfm_dynamic.bd -dn dr -dp /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/sys_link/_sysl/.xsd
INFO: [CF2BD 82-28] cf_xsd finished successfully
INFO: [SYSTEM_LINK 82-37] [14:30:35] cf2bd finished successfully
Time (s): cpu = 00:00:02 ; elapsed = 00:00:02 . Memory (MB): peak = 1402.066 ; gain = 0.000 ; free physical = 50154 ; free virtual = 339204
INFO: [v++ 60-1441] [14:30:35] Run run_link: Step system_link: Completed
Time (s): cpu = 00:00:12 ; elapsed = 00:00:13 . Memory (MB): peak = 1436.730 ; gain = 0.000 ; free physical = 50185 ; free virtual = 339231
INFO: [v++ 60-1443] [14:30:35] Run run_link: Step cf2sw: Started
INFO: [v++ 60-1453] Command Line: cf2sw -sdsl /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/ID_DNN

[15:03:41] Phase 1.4 Constrain Clocks/Macros
[15:03:41] Phase 2 Global Placement
[15:03:41] Phase 2.1 Floorplanning
[15:04:12] Phase 2.1.1 Partition Driven Placement
[15:04:12] Phase 2.1.1.1 PBP: Partition Driven Placement
[15:04:43] Phase 2.1.1.2 PBP: Clock Region Placement
[15:04:43] Phase 2.1.1.3 PBP: Compute Congestion
[15:05:14] Phase 2.1.1.4 PBP: UpdateTiming
[15:05:14] Phase 2.1.1.5 PBP: Add part constraints
[15:05:45] Phase 2.2 Global Placement Core
[15:10:24] Phase 2.2.1 Physical Synthesis In Placer
[15:11:57] Phase 3 Detail Placement
[15:11:57] Phase 3.1 Commit Multi Column Macros
[15:11:57] Phase 3.2 Commit Most Macros & LUTRAMs
[15:12:28] Phase 3.3 Area Swap Optimization
[15:12:28] Phase 3.4 Pipeline Register Optimization
[15:12:28] Phase 3.5 IO Cut Optimizer
[15:13:30] Phase 3.6 Fast Optimization
[15:14:01] Phase 3.7 Small Shape DP
[15:14:01] Phase 3.7.1 Small Shape Clustering
[15:14:32] Phase 3.7.2 Flow Legalize Slice Clusters
[15:14:32] Phase 3.7.3 Slice Area Swap
[15:15

XRT Build Version: 2.6.0 (Vitis)
       Build Date: 2020-05-07 15:30:09
          Hash ID: 9d35aca9e6be09a5402ec036a5607d26e74e01cc
Creating a default 'in-memory' xclbin image.

Section: 'DEBUG_IP_LAYOUT'(9) was successfully added.
Size   : 440 bytes
Format : JSON
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/debug_ip_layout.rtd'

Section: 'BITSTREAM'(0) was successfully added.
Size   : 40828842 bytes
Format : RAW
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/ID_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/partial.bit'

Section: 'MEM_TOPOLOGY'(6) was successfully added.
Format : JSON
File   : 'mem_topology'

Section: 'IP_LAYOUT'(8) was successfully added.
Format : JSON
File   : 'ip_layout'

Section: 'CONNECTIVITY'(7) was successfully added.
Format : JSON
File   : 'connectivity'

Section: 'CLOCK_

In [8]:
############################## Pass quantized calibration DNN model through hls4ml ##############################

compl = True
synth = True
if compl:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    cal_QDNN_hls_cfg = hls4ml.utils.config_from_keras_model(QDNNcalib, granularity='name')
    cal_QDNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    cal_QDNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in cal_QDNN_hls_cfg['LayerName'].keys():
        cal_QDNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Resources'
        cal_QDNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        cal_QDNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    cal_QDNN_hls_cfg['LayerName']['middleMan']['Precision'] = 'ap_fixed<11,6>'

    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['weight'] = 'ap_fixed<6,1>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['accum'] = 'ap_fixed<16,5>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['result'] = 'ap_fixed<15,7>'
    cal_QDNN_hls_cfg['LayerName']['RELU_CALlayer1']['Precision'] = 'ap_fixed<9,6>'

    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['weight'] = 'ap_fixed<3,1>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['accum'] = 'ap_fixed<16,7>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['result'] = 'ap_fixed<12,8>'
    cal_QDNN_hls_cfg['LayerName']['RELU_CALlayer2']['Precision'] = 'ap_fixed<9,7>'

    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['weight'] = 'ap_fixed<2,1>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['accum'] = 'ap_fixed<12,9>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['result'] = 'ap_fixed<11,10>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Strategy'] = 'Stable'

    print(cal_QDNN_hls_cfg)

    QDNNcalib_hls = hls4ml.converters.convert_from_keras_model(QDNNcalib,
                                                               hls_config=cal_QDNN_hls_cfg,
                                                               output_dir=indir+'/CAL_DNNmodel_HLS_XCU200deployment/',
                                                               backend='VivadoAccelerator',
                                                               board='alveo-u200',
                                                               io_type='io_parallel',
                                                               platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QDNNcalib_hls.compile()

if synth:
    QDNNcalib_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=True)

Interpreting Model
Topology:
Layer name: middleMan, layer type: Input
Layer name: CALlayer1, layer type: QDense
  -> Activation (linear), layer name: CALlayer1
Layer name: RELU_CALlayer1, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CALlayer1
Layer name: CALlayer2, layer type: QDense
  -> Activation (linear), layer name: CALlayer2
Layer name: RELU_CALlayer2, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CALlayer2
Layer name: CALout, layer type: QDense
  -> Activation (linear), layer name: CALout
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'middleMan': {'Precision': 'ap_fixed<11,6>', 'Strategy': 'Resources', 'ReuseFactor': 1, 'Trace': True}, 'CALlayer1': {'Precision': {'weight': 'ap_fixed<6,1>', 'accum': 'ap_fixed<16,5>', 'result': 'ap_fixed<15,7>'}, 'ReuseFactor': 1, 'Strategy': 'Resources', 'Trace': True}, 'CALlayer1_linear': {'Precision': 'ap_fixed<16,6>', 'Reuse

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'ResetAccum' (firmware/nnet_utils/nnet_dense_latency.h:68) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<15, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 64.
INFO: [HLS 200-489] Unrolling loop 'Accum1' (firmware/nnet_utils/nnet_dense_latency.h:73) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<15, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 34.
INFO: [HLS 200-489] Unrolling loop 'Accum2' (firmware/nnet_utils/nnet_dense_latency.h:74) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<15, 7, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely with a factor of 64.
INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_dense_latency.h:81) in function 'nnet::dense_latency<ap_fixed<11, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<15, 7, (ap_q_mode)5, (ap_o_mode)3

INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0' at call site (firmware/nnet_utils/nnet_dense.h:46->firmware/myproject.cpp:65) by setting 'biases[3].V' to 'b4.V.3'.
INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0.0' at call site (firmware/nnet_utils/nnet_dense.h:46->firmware/myproject.cpp:65) by setting 'biases[4].V' to 'b4.V.4'.
INFO: [XFORM 203-622] Instantiat

INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0' at call site (firmware/nnet_utils/nnet_dense.h:46->firmware/myproject.cpp:65) by setting 'biases[19].V' to 'b4.V.19'.
INFO: [XFORM 203-622] Instantiating function 'nnet::dense_latency<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0'(firmware/nnet_utils/nnet_dense_latency.h:48:27) to 'nnet::dense_latency<ap_fixed<9, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<12, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config4>.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0' at call site (firm

...

INFO: [Project 1-571] Translating synthesized netlist
Netlist sorting complete. Time (s): cpu = 00:00:00.26 ; elapsed = 00:00:00.27 . Memory (MB): peak = 3249.102 ; gain = 0.000 ; free physical = 39992 ; free virtual = 329168
INFO: [Netlist 29-17] Analyzing 3285 Unisim elements for replacement
INFO: [Netlist 29-28] Unisim Transformation completed in 0 CPU seconds
INFO: [Project 1-570] Preparing netlist for logic optimization
INFO: [Opt 31-138] Pushed 0 inverter(s) to 0 load pin(s).
Netlist sorting complete. Time (s): cpu = 00:00:00.01 ; elapsed = 00:00:00 . Memory (MB): peak = 3305.125 ; gain = 0.000 ; free physical = 39877 ; free virtual = 329054
INFO: [Project 1-111] Unisim Transformation Summary:
  A total of 434 instances were transformed.
  BUFG => BUFGCE: 1 instance 
  DSP48E2 => DSP48E2 (DSP_ALU, DSP_A_B_DATA, DSP_C_DATA, DSP_MULTIPLIER, DSP_M_DATA, DSP_OUTPUT, DSP_PREADD, DSP_PREADD_DATA): 54 instances
  IBUF => IBUF (IBUFCTRL, INBUF): 379 instances

INFO: [Common 17-83] Releas

INFO: [IP_Flow 19-2181] Payment Required is not set for this core.
INFO: [IP_Flow 19-2187] The Product Guide file is missing.
INFO: [Ipptcl 7-1486] check_integrity: Integrity check passed.
INFO: [Common 17-206] Exiting Vivado at Sat Jul  8 16:16:17 2023...
Option Map File Used: '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/data/vitis/vpp/optMap.xml'

****** v++ v2020.1 (64-bit)
  **** SW Build 2902540 on Wed May 27 19:54:35 MDT 2020
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ link can be found at:
	Reports: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CAL_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/reports/link
	Log files: /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CAL_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/logs/link
INFO: [v++ 60-1657] Initializing dispatch client.
Runnin

INFO: [CF2BD 82-28] cf2xd finished successfully
INFO: [CF2BD 82-31] Launching cf_xsd: cf_xsd -disable-address-gen -bd pfm_dynamic.bd -dn dr -dp /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CAL_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/sys_link/_sysl/.xsd
INFO: [CF2BD 82-28] cf_xsd finished successfully
INFO: [SYSTEM_LINK 82-37] [16:17:01] cf2bd finished successfully
Time (s): cpu = 00:00:02 ; elapsed = 00:00:02 . Memory (MB): peak = 1402.066 ; gain = 0.000 ; free physical = 50050 ; free virtual = 339210
INFO: [v++ 60-1441] [16:17:01] Run run_link: Step system_link: Completed
Time (s): cpu = 00:00:12 ; elapsed = 00:00:13 . Memory (MB): peak = 1436.730 ; gain = 0.000 ; free physical = 50080 ; free virtual = 339235
INFO: [v++ 60-1443] [16:17:01] Run run_link: Step cf2sw: Started
INFO: [v++ 60-1453] Command Line: cf2sw -sdsl /grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CAL_D

[16:34:48] Top-level synthesis in progress.
[16:35:19] Top-level synthesis in progress.
[16:35:38] Run vpl: Step synth: Completed
[16:35:38] Run vpl: Step impl: Started
[16:45:58] Finished 2nd of 6 tasks (FPGA linking synthesized kernels to platform). Elapsed time: 00h 28m 52s 

[16:45:58] Starting logic optimization..
[16:47:31] Phase 1 Retarget
[16:48:02] Phase 2 Constant propagation
[16:48:02] Phase 3 Sweep
[16:49:04] Phase 4 BUFG optimization
[16:49:04] Phase 5 Shift Register Optimization
[16:49:35] Phase 6 Post Processing Netlist
[16:50:37] Finished 3rd of 6 tasks (FPGA logic optimization). Elapsed time: 00h 04m 38s 

[16:50:37] Starting logic placement..
[16:51:08] Phase 1 Placer Initialization
[16:51:08] Phase 1.1 Placer Initialization Netlist Sorting
[16:52:40] Phase 1.2 IO Placement/ Clock Placement/ Build Placer Device
[16:54:13] Phase 1.3 Build Placer Netlist Model
[16:55:46] Phase 1.4 Constrain Clocks/Macros
[16:56:17] Phase 2 Global Placement
[16:56:17] Phase 2.1 Floorplan

XRT Build Version: 2.6.0 (Vitis)
       Build Date: 2020-05-07 15:30:09
          Hash ID: 9d35aca9e6be09a5402ec036a5607d26e74e01cc
Creating a default 'in-memory' xclbin image.

Section: 'DEBUG_IP_LAYOUT'(9) was successfully added.
Size   : 440 bytes
Format : JSON
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CAL_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/debug_ip_layout.rtd'

Section: 'BITSTREAM'(0) was successfully added.
Size   : 41378926 bytes
Format : RAW
File   : '/grid_mnt/data__data.polcms/cms/motta/Phase2L1T/2023_07_06_v22QNTZD/TauMinator_CE_cltw5x9_Training/CAL_DNNmodel_HLS_XCU200deployment/xclbin_files/_x/link/int/partial.bit'

Section: 'MEM_TOPOLOGY'(6) was successfully added.
Format : JSON
File   : 'mem_topology'

Section: 'IP_LAYOUT'(8) was successfully added.
Format : JSON
File   : 'ip_layout'

Section: 'CONNECTIVITY'(7) was successfully added.
Format : JSON
File   : 'connectivity'

Section: 'CLOC